# Data Cleaning and initial EDA Workbook for Project 2

---

The purpose of this workbook is to load the raw data for the project and to clean variables.  It is set up so that if new variables need attention, it starts from the raw data and produces all the changes from the beginning.  In this way, it is clear that our source is the initial raw file.  

I flipped between this workbook and another, more EDA focused workbook.  As EDA discovered items to look at, features to build, etc, I modified the datafiles accordingly.  

At the end of the notebook, csv files are generated so that other notebooks can pick up the cleaned data and use them as required.  

Input:  test.csv and train.csv  
Output:  test_nonulls#.csv and train_nonulls#.csv

The variable `out_numb` should be updated each time a new data file is generated.

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math

pd.options.display.max_columns = 999             # Allows viewing of more rows and columns in a dataframe
pd.options.display.max_rows = 999

% matplotlib inline

In [ ]:
testfile = './datasets/test.csv'
df_test = pd.read_csv(testfile)

trainfile = './datasets/train.csv'
df = pd.read_csv(trainfile)

out_numb = '6'
outfile_train = './datasets/train_nonulls' + out_numb + '.csv'
outfile_test = './datasets/test_nonulls' + out_numb + '.csv'

#sample_out = './sample_sub_reg.csv'
#df_out = pd.read_csv(sample_out)

#print(len(df), len(df_test), len(df_out))

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

#### Lot Frontage 
Lot frontage should not be 0.  To estimate a value for lot frontage when missing, I am assuming that a relationship exists between the amount of 'Lot Frontage' and the 'Lot Area'.  Below, I find the mean of the various ratios of Lot Frontage to Lot Area.  I then apply that average to the Lot Area in the case that Lot Frontage is not available.

There were two different ways to estimatet this amount.  I used 0.0079 as the estimate and sensitivity tested using 0.0071.  There was not a material difference in my results based on the sensitivity test.  

In [ ]:
print(df[df['Lot Frontage'].notnull()]['Lot Frontage'].sum() / (df[df['Lot Frontage'].notnull()]['Lot Area'].sum()))
print(np.mean(df[df['Lot Frontage'].notnull()]['Lot Frontage'] / df[df['Lot Frontage'].notnull()]['Lot Area']))

df.loc[df['Lot Frontage'].isnull(), 'Lot Frontage'] = df[df['Lot Frontage'].isnull()]['Lot Area']*0.0076
df_test.loc[df_test['Lot Frontage'].isnull(), 'Lot Frontage'] = df_test[df_test['Lot Frontage'].isnull()]['Lot Area']*0.0076

#### Alley

The data dictionary allows NA as a response indicating no alley -- change this to a more descriptive NoAlley

In [ ]:
df.loc[df['Alley'].isnull(), 'Alley'] = 'NoAlley'
df_test.loc[df_test['Alley'].isnull(), 'Alley'] = 'NoAlley'


#### Masonry Type and Area

The data dictionary allows None as a response indicating no masonry -- change this to a more descriptive NoMasonryType and 0 (for Mas Vnr Area).

In [ ]:
df.loc[df['Mas Vnr Type'].isnull(), 'Mas Vnr Type'] = 'NoMasonryType'
df.loc[df['Mas Vnr Area'].isnull(), 'Mas Vnr Area'] = 0
df_test.loc[df_test['Mas Vnr Type'].isnull(), 'Mas Vnr Type'] = 'NoMasonryType'
df_test.loc[df_test['Mas Vnr Area'].isnull(), 'Mas Vnr Area'] = 0


#### Basement fields.  

Note first need to record no basement values and for exposure and type 2.  There will be some additional work after that. Again, n/a or none was a valid entry that caused python to interpret this as a null.  For numerical values, I assumed 0 and for other cases, I created a 'NoB' identifier meaning 'No Basement'

Bsmt Qual            55  
Bsmt Cond            55  
Bsmt Exposure        58  
BsmtFin Type 1       55  
BsmtFin SF 1          1  
BsmtFin Type 2       56  
BsmtFin SF 2          1  
Bsmt Unf SF           1  
Total Bsmt SF         1  
Bsmt Full Bath        2  
Bsmt Half Bath        2  

Assumed that if a BsmtFin Type 2 is null, that it could use the value in BsmtFin Type 1.

In [ ]:
df.loc[df['Bsmt Qual'].isnull(), ['Bsmt Qual', 
                                  'Bsmt Cond', 
                                  'Bsmt Exposure', 
                                  'BsmtFin Type 1',
                                  'BsmtFin Type 2']] = 'NoB'     # No basement

df.loc[df['Bsmt Exposure'].isnull(), 'Bsmt Exposure'] = 'NoEx'
df.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 2'] = df.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 1']
df.loc[df['BsmtFin SF 1'].isnull(), 'BsmtFin SF 1'] = 0
df.loc[df['BsmtFin SF 2'].isnull(), 'BsmtFin SF 2'] = 0
df.loc[df['Bsmt Unf SF'].isnull(), 'Bsmt Unf SF'] = 0
df.loc[df['Total Bsmt SF'].isnull(), 'Total Bsmt SF'] = 0


df_test.loc[df_test['Bsmt Qual'].isnull(), ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1','BsmtFin Type 2']] = 'NoB'
df_test.loc[df_test['Bsmt Exposure'].isnull(), 'Bsmt Exposure'] = 'NoEx'
df_test.loc[df_test['BsmtFin Type 2'].isnull(), 'BsmtFin Type 2'] = df_test.loc[df['BsmtFin Type 2'].isnull(), 'BsmtFin Type 1']
df_test.loc[df_test['BsmtFin SF 1'].isnull(), 'BsmtFin SF 1'] = 0
df_test.loc[df_test['BsmtFin SF 2'].isnull(), 'BsmtFin SF 2'] = 0
df_test.loc[df_test['Bsmt Unf SF'].isnull(), 'Bsmt Unf SF'] = 0
df_test.loc[df_test['Total Bsmt SF'].isnull(), 'Total Bsmt SF'] = 0

df.loc[df['Bsmt Full Bath'].isnull(), 'Bsmt Full Bath'] = 0
df.loc[df['Bsmt Half Bath'].isnull(), 'Bsmt Half Bath'] = 0

df_test.loc[df['Bsmt Full Bath'].isnull(), 'Bsmt Full Bath'] = 0
df_test.loc[df['Bsmt Half Bath'].isnull(), 'Bsmt Half Bath'] = 0

#### Fireplace Quality 

The data dictionary allows an n/a for fireplace quality.  

I created a new value 'NoF' for 'No Fireplace' in these situations.   



In [ ]:
df.loc[df['Fireplace Qu'].isnull(), 'Fireplace Qu'] = 'NoF'
df_test.loc[df_test['Fireplace Qu'].isnull(), 'Fireplace Qu'] = 'NoF'

#### Garage variables

The data dictionary allows N/A for no garage  

For most, the values should be set to none.  I used 'NoGarage' for text columns and 0 for numerical columns.  
For the item that's left, I imputed values based on averages / most prevalent

- Garage Type         113
- Garage Yr Blt       114
- Garage Finish       114
- Garage Cars           1
- Garage Area           1
- Garage Qual         114
- Garage Cond         114

In [ ]:
df.loc[df['Garage Type'].isnull(), ['Garage Yr Blt']] = 0
df.loc[df['Garage Type'].isnull(), ['Garage Type', 'Garage Finish', 'Garage Qual','Garage Cond']] = 'NoGarage'

df_test.loc[df['Garage Type'].isnull(), ['Garage Yr Blt']] = 0       
df_test.loc[df_test['Garage Type'].isnull(), ['Garage Type', 'Garage Finish', 'Garage Qual','Garage Cond']] = 'NoGarage'

--- 

For one record, there are not values even though (through EDA), there was a detached garage.  It doesn't really matter what we put here since it's only one record, I decided to query the other records and put in the mean value of the population.  In this case, year build = 1962, cars = 2 and square footage = 420.  Since there is a garage but it is not recorded, I assumed "low quality" metrics for the text variables (unfinished and TA).

---


In [ ]:
print(df[df['Garage Type']=='Detchd']['Garage Yr Blt'].mean())
print(df[df['Garage Type']=='Detchd']['Garage Cars'].mean())
print(df[df['Garage Type']=='Detchd']['Garage Area'].mean())
#round these estimates and use to fill in missing data

In [ ]:
df.loc[df['Garage Yr Blt'].isnull(), 'Garage Yr Blt'] = 1962
df.loc[df['Garage Finish'].isnull(), 'Garage Finish'] = 'Unf'
df.loc[df['Garage Cars'].isnull(), 'Garage Cars'] = 2
df.loc[df['Garage Area'].isnull(), 'Garage Area'] = 420
df.loc[df['Garage Qual'].isnull(), 'Garage Qual'] = 'TA'
df.loc[df['Garage Cond'].isnull(), 'Garage Cond'] = 'TA'


df_test.loc[df_test['Garage Yr Blt'].isnull(), 'Garage Yr Blt'] = 1962
df_test.loc[df_test['Garage Finish'].isnull(), 'Garage Finish'] = 'Unf'
df_test.loc[df_test['Garage Cars'].isnull(), 'Garage Cars'] = 2
df_test.loc[df_test['Garage Area'].isnull(), 'Garage Area'] = 420
df_test.loc[df_test['Garage Qual'].isnull(), 'Garage Qual'] = 'TA'
df_test.loc[df_test['Garage Cond'].isnull(), 'Garage Cond'] = 'TA'

#### Remaining null values

- Pool QC            2042
- Fence              1651
- Misc Feature       1986
- Electrical            1

These each have N/A as an option and they were all text.  I used a new category indicating "NoXYZ" instead of the data dictionary N/A.

In [ ]:
df.loc[df['Pool QC'].isnull(), 'Pool QC'] = 'NoPool'
df.loc[df['Fence'].isnull(), 'Fence'] = 'NoFence'
df.loc[df['Misc Feature'].isnull(), 'Misc Feature'] = 'NoMiscF'

df_test.loc[df_test['Pool QC'].isnull(), 'Pool QC'] = 'NoPool'
df_test.loc[df_test['Fence'].isnull(), 'Fence'] = 'NoFence'
df_test.loc[df_test['Misc Feature'].isnull(), 'Misc Feature'] = 'NoMiscF'
df_test.loc[df_test['Electrical'].isnull(), 'Electrical'] = 'SBrkr'

#### New Features

---

I thought it might be instructive to use the maximum of the year built, the year remodeled, or garage added as a potential indication of when substantive improvements to the property were enacted.  New varialbe 'Yr Latest Change' added to the data frame.

---


In [ ]:
df['Yr Latest Change'] = df[['Garage Yr Blt','Year Built', 'Year Remod/Add', ]].max(axis = 1)
df_test['Yr Latest Change'] = df_test[['Garage Yr Blt','Year Built', 'Year Remod/Add', ]].max(axis = 1)

--- 

The neighborhood variable is likely not usefull with as many neighborhoods involved.  I decided to group the neighborhoods by average sales price to create new categories.

---

In [ ]:
df['Neighborhood'].value_counts()

In [ ]:
def get_neigh_cat(list_in):
    list_out = []
    cat1 = ['StoneBr','NridgHt','NoRidge']
    cat2 = ['GrnHill','Veenker','Timber']
    cat3 = ['Somerst','ClearCr','Crawfor','CollgCr', 'Blmngtn']
    cat4 = ['NWAmes','Gilbert','Greens','SawyerW']
    cat5 = ['Mitchel','NAmes','Blueste']
    cat6 = ['NPkVill','Sawyer','Landmrk','SWISU','Edwards','BrkSide', 'OldTown']
    cat7 = ['BrDale','IDOTRR','MeadowV']
    for each in list_in:
        if each in cat1:
            list_out.append('cat1')
        elif each in cat2:
            list_out.append('cat2')
        elif each in cat3:
            list_out.append('cat3')
        elif each in cat4:
            list_out.append('cat4')
        elif each in cat5:
            list_out.append('cat5')
        elif each in cat6:
            list_out.append('cat6')
        elif each in cat7:
            list_out.append('cat7')
        else:
            list_out.append(each)
    return(list_out)
            

In [ ]:
df['Neighb feat'] = get_neigh_cat(df['Neighborhood'])
df_test['Neighb feat'] = get_neigh_cat(df_test['Neighborhood'])

---

For this iteration of data files, I have decided to take the log of certain items that are skewed right.  Lot frontage, lot area, gr living area, and total bsmt SF.  In the modeling notebook, I will make two other changes, MS SubClass turned into category variable, and exclude other SF variables.  For log function, need the python function that allows log(0)

---

In [ ]:
#var_list = ['Lot Frontage', 'Lot Area', 'Gr Liv Area', 'Total Bsmt SF']

#for each in var_list:
#    df[each] = np.log1p(df[each])
#    df_test[each] = np.log1p(df[each])


In [ ]:
print(sum(df.isnull().sum()))
print(sum(df_test.isnull().sum()))

#df_test.isnull().sum()

## Output cleansed data into new data file

In [ ]:
df.to_csv(outfile_train, index = False)
df_test.to_csv(outfile_test, index = False)

In [ ]:
df.head()